In [22]:
import struct
from datetime import datetime
import binascii

filename="G:/CSU3601C.bin"
with open(filename,'rb') as f:
    decrypted_log = f.read()
filesize=len(decrypted_log)

candump_list = []
message_list = []

for block_start in range(0,filesize,512):
    block = decrypted_log[block_start:block_start+508]
    recorded_block_crc = struct.unpack('<L',decrypted_log[block_start+508:block_start+512])[0]
    computed_block_crc = binascii.crc32(block)
    if recorded_block_crc != computed_block_crc:
        print("CRC didn't match")
        continue
    if block[0:4]!=b'CAN2':
        print("Not CAN2 Format")
        print(f"{block_start}: {block}")
        continue
    #Add some meta data parsers here
    
    for recordNum in range(19): #Parse through CAN message
        record = block[4+recordNum*25:4+(recordNum+1)*25]
        channel = record[0]
        timeSeconds = struct.unpack("<L",record[1:5])[0]
        timeMicrosecondsAndDLC = struct.unpack("<L",record[13:17])[0]
        timeMicroseconds = timeMicrosecondsAndDLC & 0x00FFFFFF
        DLC = (timeMicrosecondsAndDLC & 0xFF000000)>>24
        abs_time = timeSeconds + timeMicroseconds * 0.000001
        time_str = datetime.fromtimestamp(abs_time, tz=None).isoformat()
        ID = struct.unpack("<L",record[9:13])[0]
        message_bytes = record[17:25]
        #create list for all data parsed
        candump_list.append("({:0.6f}) can{:0.0f} {:08X}#{}"
                            .format(abs_time,channel,ID,''.join(["{:02X}"
                                .format(b) for b in message_bytes])))
        message_list.append(["{:0.6f}".format(abs_time),"can{:0.0f}".format(channel),"{:08X}"
                            .format(ID),]+["{:02X}"
                                .format(b) for b in message_bytes])

with open(filename+"_candump.log", 'w') as f:
    for line in candump_list:
        f.write(line+'\n')

with open(filename+".csv", 'w') as f:
    for line in message_list:
        f.write(','.join(line)+'\n')
print(f"Finished {filename} of {filesize} bytes")

Not CAN2 Format
54512128: b'\xff\xff\xff\xff\x003\xc9\xc9b4\xa6L:\x03\xfe\xff\x0c\xf4\xab\t\x08\x8e}\x8fA\x00\x00\x7f\xe1\x003\xc9\xc9bX\xa7L:"\xfb\xff\x0c\xf9\xac\t\x08\x00f\xff\xff\xff\xff\xff\xff\x003\xc9\xc9bx\xa8L:\x95G\xfe\x186\xae\t\x08\x7f\xff\xff\xff\xff\xff\xff\xff\x003\xc9\xc9b\x9e\xa9L:"\xfa\xff\x0c>\xaf\t\x08\x011\xff\xff\xff\xff\xff\xff\x003\xc9\xc9b\xbe\xaaL:G\xf3\xff\x18{\xb0\t\x08"\xff\xfc\xff\xff\xff\xff\xff\x003\xc9\xc9b\xdc\xabL:"\xfb\xff\x0c|\xb1\t\x08\x00j\xa6\x0e\xff\xff\xff\xff\x003\xc9\xc9bn\xb7L:\x00\x04\xf0\x0c+\xbd\t\x08\xfe\xff}\x00\x00\xff\xf4\xff\x003\xc9\xc9b\x91\xb8L:\x05\xff\xef\x08/\xbe\t\x08n\xff\xff\x00\xff\xff\xff\xff\x013\xc9\xc9b\x7f\xb9L:\xf0\x12\xfe\x0c>\xbf\t\x08}}\xf0?\xff\xff\xff\xff\x003\xc9\xc9b\xb3\xb9L:\x00\x92\xfd\x0cQ\xbf\t\x08\xf0\xff\xff\xff\xff\xff\xff\xff\x003\xc9\xc9b\xcb\xbaL:\x05\xfe\xff\x08\x87\xc0\t\x08\x8b\x86\xff\xf1\xb0\x04H=\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff